## Nth Largest

-- use cases include 0 and -1

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os

In [2]:
%load_ext sql
%sql postgresql://toofanmacpro@localhost:5432/testdb


### My Pandas solution

In [3]:
id = [1,2,3]
salary = [100,200,300]

df = pd.DataFrame({'id':id, 'salary': salary})

In [4]:
def nLargest(daa: pd.DataFrame, N: int) -> pd.DataFrame: # '->' indicates what the output should be, and ':' defines what the inputs should be
    if N <= 0 : 
        return (pd.DataFrame({"SecondHighestSalary": [None]}, index = [0]))

    unique = (daa
          .sort_values(by = 'salary', ascending=False)
          .salary
          .drop_duplicates()
          .nlargest(N)
          )

    print(unique)
    
    if (len(unique) < N):
        return (pd.DataFrame({"SecondHighestSalary": [None]}, index = [0]))
    else:
        return (pd.DataFrame({"SecondHighestSalary":[unique.iloc[-1]]}, index = [0]))

In [5]:
nLargest(df,2)

2    300
1    200
Name: salary, dtype: int64


,SecondHighestSalary
0,200


In [6]:
nLargest(df,0)

,SecondHighestSalary
0,None


In [7]:
nLargest(df,-10)

,SecondHighestSalary
0,None


###  MY SQL Query of NthLargest

In [8]:
%%sql
SELECT 
    DISTINCT salary
FROM "Employee"
ORDER BY salary DESC
LIMIT 1 OFFSET 1;

 * postgresql://toofanmacpro@localhost:5432/testdb
1 rows affected.


salary
200


In [9]:
%%sql
CREATE OR REPLACE FUNCTION NthLargest(N INT) RETURNS TABLE (salary BIGINT) AS $$
BEGIN
RETURN QUERY (
    SELECT
        CASE WHEN N <= 0 THEN NULL
        ELSE 
            (
                SELECT 
                    DISTINCT e.salary
                FROM "Employee" AS e
                ORDER BY e.salary DESC
                LIMIT 1 OFFSET (N-1)
            )
        END AS salary
    );
END;
$$ LANGUAGE plpgsql;

 * postgresql://toofanmacpro@localhost:5432/testdb
Done.


[]

In [10]:
%%sql
SELECT * FROM NthLargest(1)

 * postgresql://toofanmacpro@localhost:5432/testdb
1 rows affected.


salary
300


IF a funciton has to be dropped

In [11]:
# %%sql
# DROP FUNCTION nthlargest(integer)

In [12]:
%%sql
SELECT * FROM NthLargest(-1)

 * postgresql://toofanmacpro@localhost:5432/testdb
1 rows affected.


salary
None


### Nth Highest = SQL - best solution

In [13]:
%%sql
CREATE OR REPLACE FUNCTION NthBigSalary(N INT) RETURNS TABLE(salary BIGINT) AS $$
BEGIN
    IF N < 1 THEN 
        RETURN QUERY (SELECT NULL::BIGINT AS salary);
    ELSE
        RETURN QUERY(
            SELECT 
                e.salary
            FROM "Employee" AS e
            GROUP BY e.salary
            ORDER BY e.salary DESC
            OFFSET N-1 LIMIT 1
        );
    END IF;
END;
$$ LANGUAGE plpgsql;

 * postgresql://toofanmacpro@localhost:5432/testdb
Done.


[]

we used BIGINT because thats how our data is stored

In [14]:
%%sql
SELECT * FROM NthBigSalary(2)

 * postgresql://toofanmacpro@localhost:5432/testdb
1 rows affected.


salary
200


In [15]:
%%sql
SELECT * FROM NthBigSalary(-1)

 * postgresql://toofanmacpro@localhost:5432/testdb
1 rows affected.


salary
None


In [16]:
%%sql
SELECT * FROM NthBigSalary(0)

 * postgresql://toofanmacpro@localhost:5432/testdb
1 rows affected.


salary
None
